### Import Data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("homeprices_banglore.csv")
df.head()

,area,bedrooms,price
0,1056,2,39.07
1,2600,4,120.00
2,1440,3,62.00
3,1521,3,75.00
4,1200,2,51.00


Now we have to scail those columns to (0,1) by MinMax Scaler

In [3]:
from sklearn import preprocessing

sx = preprocessing.MinMaxScaler()
sy = preprocessing.MinMaxScaler()

# we will scail 2 features "area" and "bedrooms" into a specific column
scail_X = sx.fit_transform(df.drop("price", axis= 1))

# we will scail the columns "price" into another specific column
scail_y = sy.fit_transform(df['price'].values.reshape(df.shape[0], 1))

In [4]:
scail_X.shape

(20, 2)

In [5]:
scail_y.shape 

(20, 1)

### Implement Mini-Batch Gradient Descent

In [6]:
import random

In [7]:
def mini_batch_GD(X, y_true, epochs, batch_size, learning_rate = 0.01):
    
    # Set up batch size
    number_of_features = X.shape[1] # số lượng features cụ thể ở đây đang là 2 (area and bedrooms)
    weights = np.ones(shape= (number_of_features)) # set trọng số là 1 ma trận (1,2) với giá trị ban đầu là 1
    bias = 0 # set bias = 0 vì lúc đầu chưa được tính nên chưa có gì
    total_samples = X.shape[0]
    
    cost_list = []
    epoch_list = []
    
    # Shuffle the data: shuffle the dataset at the beginning of each epoch to ensure randomnes and avoid any potential bias.
    for i in range(epochs):
        # Shuffle the data at the start of each epoch
        random_indices = np.arange(total_samples) # tạo ra 1 mảng random
        np.random.shuffle(random_indices) # xáo trộn random
        X = X[random_indices] # xáo trộn feature cột X dựa trên ind đã được random ở trên
        y_true = y_true[random_indices] # xáo trộn cột y 
        
        
        """Ở bước này ta sẽ thực hiện mini-batch: mini-batch có nghĩa là nó là vừa bgd và sgd kết hợp với nhau đầu tiên nó sẽ chọn random
        giá trị trong tập dữ liệu xong đó nó sẽ chia thành cụm với bước nhảy bằng nhau sau đó là thực hiện bgd ở mỗi batch mà nó đã chia
        đều ra 
        Example: Chúng ta có 100 samples thì cái batch_size ở đây là số bước nhảy ví dụ tôi chọn 10 thì nó sẽ chia 100 samples ra làm 10 phần
        bằng nhau sau đó là thực hiện bgd trong mỗi 10 batchs đó và cập nhật weights và bias liên tục. Đó là lí do vì sao nó tối ưu hơn 2 thằng
        trước bgd và sgd vì nó là sự kết hợp tính chất của 2 thằng
        """
        # Iterate each mini-batch
        for j in range(0, total_samples, batch_size):
            # Định nghĩa mini-batch
            X_batch = X[j: j+batch_size] #(giá trị chạy từ 0-> bước nhảy đã set)
            y_batch = y_true[j: j+batch_size]
            
            y_pred = np.dot(weights, X_batch.T) + bias # formula của y_pred = weight*x + bias
            
            w_derivative = -(2/ X_batch.shape[0]) * (X_batch.T * (y_batch - y_pred)) # đạo hàm riêng theo weights
            """lưu ý: tại sao ở đây mình lại chia cho X_batch.shape[0] là vì nó đã được phân ra mỗi khung nhỏ từ đâu đến đâu như nhau thì mình sẽ tính
            đạo hàm riêng cho mỗi ô dữ liệu đó thôi"""
            b_derivative = -(2/ X_batch.shape[0]) * np.sum(y_batch - y_pred) # đạo hàm riêng theo bias
            
            # Cập nhật trọng số và bias sử dụng gradient
            weights = weights - learning_rate * w_derivative
            bias = bias - learning_rate * b_derivative
            
        
        # Calculate cost after every epoch and update y_pred all of dataset
        y_pred_all = np.dot(weights, X.T) + bias 
        cost = np.mean(np.square(y_true.flatten() - y_pred_all)) # MSE function
        
        # Store the cost every 10 epochs
        if i%10 == 0:
            cost_list.append(cost)
            epoch_list.append(i)
            
    return weights, bias, cost, cost_list, epoch_list

In [8]:
w, b, cost, cost_list, epoch_list = mini_batch_GD(scail_X, scail_y.reshape(scail_y.shape[0], 1), 100, 2)

In [9]:
w_reshape = np.array([w.reshape(-1)[0], w.reshape(-1)[1]])

In [10]:
w_reshape

array([0.91915486, 0.04114171])

In [11]:
cost_list, epoch_list

([0.1311747367652836,
  0.023550030671916522,
  0.015116163926887648,
  0.010424006027181482,
  0.007912161520839501,
  0.006859285921293817,
  0.007136341543772992,
  0.008081621667185778,
  0.009802367372188866,
  0.011936622761444813],
 [0, 10, 20, 30, 40, 50, 60, 70, 80, 90])

In [12]:
def predict_price(area, bedrooms, w, bias):
    scaled_features = sx.transform([[area, bedrooms]]) # scaled_features có kích thước (1, 2)
    
    # Tính giá trị dự đoán (trên dữ liệu đã chuẩn hóa)
    scaled_price = w[0]*scaled_features[0][0] + w[1]*scaled_features[0][1] + bias
    
    # Đảo ngược quá trình chuẩn hóa để lấy giá trị giá gốc
    price = sy.inverse_transform([[scaled_price]])[0][0] # Trả về giá trị gốc của giá nhà
    return price

In [13]:
predict_price(2785, 4, w_reshape, b)

d:\Python Download\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


115.48971542574604